# **DenseNet-169 for multi-class classification of CXR images**

### Objectives

The aim of this notebook is to train a selected pretrained network [TBD] on the Chest X-ray (Pneumonia) data set sourced from Kaggle. The images are composed of CXR scans representing normal lungs, bacterial pneumonia, and viral pneumonia.

A **classification head** will be built on top of the chosen network, in which various architectures are developed and evaluated. Several preprocessing techniques are applied, each of which will be evaluated based on the model performance when trained on them. Once trained and evaluated, each version of the network will be further evaluated with the **GRAD-CAM** framework to visualise the regions of importance in the images.

The experiment tracking will be conducted with the *Weights and Biases* platform. 

### Machine Configurations

`GPU` : NVIDIA GeForce RTX 4090;
`CPU` : AMD Ryzen 7 3700X 8-Core Processor;
`VRAM` : 24.0 GB

### Load all required libraries:

In [2]:
import os
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import f1_score, matthews_corrcoef

import tensorflow as tf

# Suppress TensorFlow messages:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print("TensorFlow version : ", tf.__version__)  # check TensorFlow version
gpu = tf.config.list_physical_devices("GPU")    # check if TensorFlow is using the GPU
print(gpu)

# Enable memory growth for GPU:
try:
   tf.config.experimental.set_memory_growth(gpu[0], True)
   print("Memory growth set.")
except:
    print("GPU runtime already initialised.")

TensorFlow version :  2.10.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Configure experiment:

In [ ]:
# Configure notebook name for WandB:
%env "WANDB_NOTEBOOK_NAME" "PCXR-evaluating-pretrained-cnn-model"
import wandb
from wandb.keras import WandbMetricsLogger
wandb.login()

In [7]:
# Choose method for hyperparameter selection:
sweep_config = {
    "method": "bayes"
    }

In [9]:
# Establish objective to optimise:
metric = {
    "name": "val_loss",
    "goal": "minimize"   
    }

sweep_config["metric"] = metric

In [11]:
# Choose hyperparameters to sweep:
param_dict = {
    "batch_size": {
        "values": [16, 32, 64]
        },
    "optimiser": {
        "values": ["adam", "sgd"]
        },
    "fc_layer_size": {
        "values": [32, 64, 128, 256]
        },
    "dropout_1": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_2": {
        "values": [0.2, 0.6, 0.8]
        },
    "dropout_3": {
        "values": [0.2, 0.6, 0.8]
        },
    }

sweep_config['parameters'] = param_dict

In [12]:
# Instantiate constant value for epoch:
param_dict.update({
    "epochs": {
        'value': 10}
    })

In [14]:
# Use uniform distribution for learning rate:
param_dict.update({
    'learning_rate': {
        "distribution": "uniform",
        "min": 0.0001,
        "max": 0.005
      }
    })

In [15]:
import pprint

pprint.pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64]},
                'dropout_1': {'values': [0.2, 0.6, 0.8]},
                'dropout_2': {'values': [0.2, 0.6, 0.8]},
                'dropout_3': {'values': [0.2, 0.6, 0.8]},
                'epochs': {'value': 10},
                'fc_layer_size': {'values': [32, 64, 128, 256]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.005,
                                  'min': 0.0001},
                'optimiser': {'values': ['adam', 'sgd']}}}


### Load data:

In [ ]:
def build_datasets(train_dir="..\\artifacts\\train",
                   test_dir="..\\artifacts\\test",
                   class_names=["normal", "bacteria", "virus"],
                   image_size=(300, 300),
                   val_split=0.2):
    
    train_ds, valid_ds = tf.keras.utils.image_dataset_from_directory(
        directory=train_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=None,
        image_size=image_size,
        shuffle=True,
        seed=42,
        validation_split=val_split,
        subset="both")

    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=test_dir,
        label_mode="int",
        class_names=class_names,
        batch_size=None,
        image_size=image_size,
        shuffle=True,
        seed=42,
        subset=None)
    
    return (train_ds, valid_ds, test_ds)

### Construct network:

In [ ]:
def build_network(fc_layer_size=128, dropout_1=0.2, dropout_2=0.2, dropout_3=0.2):

    # Configure image shape for model input shape
    IMG_SHAPE = (300, 300)

    # Instantiate base model
    base_model = tf.keras.applications.densenet.DenseNet169(
        input_shape=IMG_SHAPE,
        include_top=False,
        weights="imagenet")

    # Freeze convolutional base
    base_model.trainable = False

    inputs = tf.keras.Input(shape=IMG_SHAPE)

    # Add an augmentation layer for additional training data
    augment_layer = tf.keras.Sequential([
        tf.keras.layers.RandomContrast(factor=0.2, seed=42),
        tf.keras.layers.RandomBrightness(factor=0.2, seed=42),
        tf.keras.layers.RandomRotation(factor=0.01, seed=42)
    ])

    # Establish model architecture:
    x = augment_layer(inputs)
    x = tf.keras.applications.densenet.preprocess_input(x)
    x = base_model(x, training=False)    # Prevent training of BN layers

    # Add custom classification head:
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_1)(x)  # Prevent overfitting
    x = tf.keras.layers.Dense(fc_layer_size, 3, activation="relu", input_shape=(1664,))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_2)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(dropout_3)(x)

    outputs = tf.keras.layers.Dense(3, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)

### Define optimiser:

In [ ]:
def build_optimiser(learning_rate=0.0001, optimiser="adam"):
    # Define optimisers:
    if optimiser.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=learning_rate)
    if optimiser.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=learning_rate)

### Create callback to log custom metrics:

In [ ]:
# Compute MCC, F1, and AUC scores and log in WandB:
class CustomLogCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, x_val, y_val):
        super().__init__()
        self.model = model
        self.x_val = x_val
        self.y_val = y_val

        # Instantiate standalone metrics:
        self._mcc = tf.keras.metrics.Mean(name="mcc")
        self._f1score = tf.keras.metrics.Mean(name="f1_score")
        self._aucroc = tf.keras.metrics.AUC(name="auc_roc")

        self.epoch = 0

    def on_epoch_end(self, epoch, logs=None):
        self.epoch += 1

        self._mcc.reset_state()
        self._f1score.reset_state()
        self._aucroc.reset_state()

        print("Generating predictions and computing metrics for Epoch {} ".format(self.epoch))
        predictions = self.model.predict(self.x_val)

        f1score = f1_score(self.y_val, np.argmax(predictions, axis=-1),
                            average=None)
        mcc = matthews_corrcoef(self.y_val, np.argmax(predictions, axis=-1))

        self._mcc.update_state(mcc)
        self._f1score.update_state(f1score)
        self._aucroc.update_state(self.y_val, np.argmax(predictions, axis=-1))

        print("training loss : {} , training acc : {} , mcc score : {}".format(
            logs["loss"], logs["accuracy"], self._mcc.result().numpy()
        ))
        print("aucroc score  : {} , f1 score     : {} ".format(
            self._aucroc.result().numpy(), self._f1score.result().numpy()
        ))

        # Log metrics to WandB:
        wandb.log({"mcc": self._mcc.result().numpy(),
                   "fmeasure": self._f1score.result().numpy(),
                   "auc_roc": self._aucroc.result().numpy()})

### Create training function:

In [ ]:
# Build data sets outside of train function
train_ds, valid_ds, _ = build_datasets()

# Convert validation data to arrays for custom metric computation:
x_val = np.concatenate([x for x, y in valid_ds], axis=0)
y_val = np.concatenate([y for x, y in valid_ds], axis=0)

def train(model, batch_size=32, epochs=10, learning_rate=0.0001, optimiser_name="adam"):

    optimiser = build_optimiser(learning_rate=learning_rate, optimiser=optimiser_name)

    # Compile model:
    model.compile(optimizer=optimiser,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=["accuracy"])
    
    # Fit model:
    # Configure callbacks:
    callbacks = [CustomLogCallback(model, x_val, y_val),
                 wandb.keras.WandbMetricsLogger()]
    # Train model:
    model.fit(train_ds, epochs=epochs, batch_size=batch_size,
              validation_data=valid_ds, callbacks=callbacks, verbose=0)

### Loop training function in sweep:

In [ ]:
def sweep_train(config_defaults=None):
    
    # Initialise wandb and start run:
    with wandb.init(config=config_defaults):
        # Specify other experimentation configurations:
        wandb.config.architecture = "DenseNet"
        wandb.config.dataset = "Chest X-Ray Images (Pneumonia)"
        wandb.config.num_classes = 3
        wandb.notes = "Sweeping through DenseNet model variations"

        model = build_network(wandb.config.fc_layer.size,
                              wandb.config.dropout_1,
                              wandb.config.dropout_2,
                              wandb.config.dropout_3)

        train(model=model, batch_size=wandb.config.batch_size,
              epochs=wandb.config.epochs, learning_rate=wandb.config.learning_rate,
              optimiser_name=wandb.config.optimiser)

### Initialise sweep and run agent:

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="PCXR-evaluating-pretrained-model-variations")

In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=15)